In [10]:
import pygame
import random
import math
import numpy as np

In [95]:
SAFETY_DISTANCE = 20
KILL_DISTANCE = 30
LAMBDA = 4

In [111]:
class Car:
    def __init__(self, next_stop = None):
        self.pos = None
        self.direction = None
        self.distance = None
        self.next = None
        self.prev = None
        self.next_stop = next_stop
        
        self.speed = 2*LAMBDA
        self.max_speed = 4*LAMBDA
        self.acceleration = 0.02*LAMBDA
        self.lookahead = self.speed * 20

        
        self.color = (255, 255, 255)
        
    def get_direction(self):
        direction_not_norm = self.next_stop.pos - self.next_stop.prev.pos
        return direction_not_norm/np.sqrt(np.sum((direction_not_norm)**2))
        
    def determine_distance(self):
        if self.next_stop.red == True:
            return min(math.dist(self.pos, self.next.pos), 
                       math.dist(self.pos, self.next_stop.pos))
        else:
            return math.dist(self.pos, self.next.pos)
    
    def step(self):
        new_distance = self.determine_distance()
        
        #kill
        if self.next_stop.last == True and new_distance < KILL_DISTANCE:
            try:
                self.prev.next = self.next_stop
            except:
                pass
            return
        
        #new stoplight
        if np.sum(np.sign(self.next_stop.pos - self.pos) + np.sign(self.direction)) == 0:
            self.next_stop = self.next_stop.next
            self.direction = self.get_direction()

        if new_distance < self.lookahead + SAFETY_DISTANCE:
            self.speed = max(self.speed/2, 0)
        else:
            self.speed = min(self.speed+self.acceleration, self.max_speed)
            
        self.distance = new_distance
        self.lookahead = self.speed * 20
        self.pos = self.pos + (self.speed*self.direction)

In [112]:
class Stoplight:
    def __init__(self, pos, red = False, fake = True, first = False, last = False):
        self.pos = np.array(pos)
        self.next = None
        self.prev = None
        self.red = red
        self.fake = fake
        self.first = first
        self.last = last
        self.get_color()
        
    def step(self):
        self.red = not self.red
        self.get_color()
        
    def get_color(self):
        if self.red == True:
            self.color = (255, 0, 0)
        else:
            self.color = (0, 255, 0)
            
    def get_last(self):
        a = self
        while a.next != None:
            a = a.next
        return a

In [113]:
#at first we need to create the stoplight linked list
class Stop_list:
    
    def __init__(self, head = None):
        self.head = head
        
    def __repr__(self):
        lst = []
        self._aid_repr(self.head, lst)
        return ' '.join(lst)
    
    def _aid_repr(self, node, lst):
        if node:
            lst.append(str(node.pos))
            self._aid_repr(node.next, lst)
    
    def append(self, other):
        if self.head == None:
            self.head = other
            return
        
        a = self.head
        while a.next != None:
            a = a.next
        other.prev = a
        a.next = other

In [114]:
#lastly the car linked list
class Car_list:
    
    def __init__(self, head = None):
        self.head = head
        
    def __repr__(self):
        lst = []
        self._aid_repr(self.head, lst)
        return ' '.join(lst)
    
    def _aid_repr(self, node, lst):
        if node:
            lst.append(str(node.pos))
            self._aid_repr(node.next, lst)
        
    def front_append(self, other):
        #appending the car
        if self.head == None:
            self.head = other
            self.head.next = other.next_stop.get_last()
            #initializing the other details of the car
            other.pos = other.next_stop.pos
            other.direction = other.get_direction()
            other.distance = other.determine_distance()
            return
        
        self.head.prev = other
        a = self.head
        other.next = a
        self.head = other
        #initializing the other details of the car
        other.pos = other.next_stop.pos
        other.direction = other.get_direction()
        other.distance = other.determine_distance()

In [116]:
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Traffic Simulation")
img = pygame.image.load("intersection.png").convert()

stl = []
cal = []

sl1 = Stop_list()
cl1 = Car_list()
sl1.append(Stoplight((420, 601), first = True))
sl1.append(Stoplight((420, 600)))
sl1.append(Stoplight((420, 320), red = True, fake = False))
sl1.append(Stoplight((420, 0), last = True))
stl.append(sl1)
cal.append(cl1)

sl2 = Stop_list()
cl2 = Car_list()
sl2.append(Stoplight((380, -1), first = True))
sl2.append(Stoplight((380, 0)))
sl2.append(Stoplight((380, 280), red = True, fake = False))
sl2.append(Stoplight((380, 600), last = True))


stl.append(sl2)
cal.append(cl2)

sl3 = Stop_list()
cl3 = Car_list()
sl3.append(Stoplight((-1, 320), first = True))
sl3.append(Stoplight((0, 320)))
sl3.append(Stoplight((380, 320), fake = False))
sl3.append(Stoplight((800, 320), last = True))


stl.append(sl3)
cal.append(cl3)

sl4 = Stop_list()
cl4 = Car_list()
sl4.append(Stoplight((801, 280), first = True))
sl4.append(Stoplight((800, 280)))
sl4.append(Stoplight((420, 280), fake = False))
sl4.append(Stoplight((0, 280), last = True))


stl.append(sl4)
cal.append(cl4)

i = 0
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    screen.fill((100, 100, 100))
    
    for sl, cl in zip(stl, cal):
        #stoplights
        stoplight = sl.head
        while stoplight != None:
            if i%400 == 0 and stoplight.fake == False:
                stoplight.step()
            if stoplight.fake == False:
                pygame.draw.circle(screen, stoplight.color, stoplight.pos, 8)
            stoplight = stoplight.next
    
        #cars
        if min(np.random.poisson(0.01), 1) == 1: #rate at which to spawn cars
            cl.front_append(Car(sl.head.next))
        if cl.head != None:
            car = cl.head
            while car.next != None:
                car.step()
                pygame.draw.circle(screen, car.color, car.pos, 4)
                car = car.next
            
    pygame.display.update()
    i += 1

KeyboardInterrupt: 

In [117]:
pygame.quit()